In [2]:
import matplotlib.pyplot as plt
import torch
# ЗАДАЧА 1: Визуализация Attention
from transformers import BertTokenizer, BertModel

# Загружаем модель с включенным выводом attention весов
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_attentions=True)
model.eval()  # переводим в режим оценки

# Токенизируем предложение
text = "The cat sat on the mat"
inputs = tokenizer(text, return_tensors="pt")

# Прямой проход через модель
with torch.no_grad():
    outputs = model(**inputs)

# В outputs.attentions лежит список тензоров для каждого из 12 слоев
# Формат: (batch_size, num_heads, seq_len, seq_len)
attentions = outputs.attentions

print(f"Количество слоев: {len(attentions)}")
print(f"Форма attention одного слоя: {attentions[0].shape}")
print(f"Количество attention heads: {attentions[0].shape[1]}")

# Посмотрим, как выглядит токенизация
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
print("\nТокены:", tokens)
print("ID токенов:", inputs['input_ids'][0].tolist())


def visualize_attention(layer_idx=0, head_idx=0, word="cat"):
    """Визуализация attention для конкретного слова"""

    # Находим индекс токена для нужного слова
    tokenized_text = tokenizer.tokenize(text)
    word_index = tokenized_text.index(word) + 1  # +1 из-за [CLS]

    # Берем attention веса из указанного слоя и head
    attention = attentions[layer_idx][0, head_idx].numpy()

    # Визуализируем
    plt.figure(figsize=(10, 8))
    plt.imshow(attention, cmap='Reds', vmin=0, vmax=1)

    # Настройки графика
    plt.xticks(range(len(tokens)), tokens, rotation=45)
    plt.yticks(range(len(tokens)), tokens)
    plt.title(f"Layer {layer_idx}, Head {head_idx}, Word: '{word}'")
    plt.colorbar()
    plt.tight_layout()
    plt.show()

    # Покажем численно, на какие токены смотрит "cat"
    print(f"\nAttention от слова '{word}' (индекс {word_index}):")
    for i, (token, weight) in enumerate(zip(tokens, attention[word_index])):
        print(f"{token:10s}: {weight:.4f}")


# Смотрим внимания для разных конфигураций
visualize_attention(layer_idx=0, head_idx=0, word="cat")
visualize_attention(layer_idx=5, head_idx=3, word="sat")


def analyze_word_attention(word="the"):
    """Анализируем, как меняется внимание к слову по слоям"""
    word_idx = tokens.index(word) if word in tokens else -1

    if word_idx == -1:
        print(f"Слово '{word}' не найдено в токенах")
        return

    all_attentions = []

    # Собираем attention веса по всем слоям и head'ам
    for layer_idx in range(len(attentions)):
        layer_attention = attentions[layer_idx][0, :, word_idx, :].mean(0)
        all_attentions.append(layer_attention.numpy())

    # Визуализируем тепловую карту по слоям
    plt.figure(figsize=(12, 8))
    plt.imshow(np.array(all_attentions), aspect='auto', cmap='Reds')
    plt.xlabel('Токены')
    plt.ylabel('Слои')
    plt.yticks(range(len(attentions)))
    plt.xticks(range(len(tokens)), tokens, rotation=45)
    plt.title(f"Эволюция внимания к слову '{word}' по слоям BERT")
    plt.colorbar()
    plt.tight_layout()
    plt.show()


analyze_word_attention("cat")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
import numpy as np


def simple_self_attention(X, d_k):
    """
    Простая реализация self-attention без обучения весов

    X: входные эмбеддинги [seq_len, d_model]
    d_k: размерность ключей/запросов
    """
    seq_len, d_model = X.shape

    # Инициализируем веса случайно (в реальной модели они обучаются)
    np.random.seed(42)
    W_q = np.random.randn(d_model, d_k) * 0.1
    W_k = np.random.randn(d_model, d_k) * 0.1
    W_v = np.random.randn(d_model, d_model) * 0.1

    # Шаг 1: Получаем Q, K, V
    Q = X @ W_q  # [seq_len, d_k]
    K = X @ W_k  # [seq_len, d_k]
    V = X @ W_v  # [seq_len, d_model]

    print(f"Q shape: {Q.shape}")
    print(f"K shape: {K.shape}")
    print(f"V shape: {V.shape}")

    # Шаг 2: Вычисляем attention scores
    attention_scores = Q @ K.T  # [seq_len, seq_len]
    print(f"\nAttention scores shape: {attention_scores.shape}")

    # Шаг 3: Масштабируем и применяем softmax
    attention_scores = attention_scores / np.sqrt(d_k)
    attention_weights = np.exp(attention_scores) / np.sum(np.exp(attention_scores), axis=-1, keepdims=True)

    print(f"Attention weights shape: {attention_weights.shape}")

    # Шаг 4: Применяем weights к значениям
    output = attention_weights @ V  # [seq_len, d_model]

    return output, attention_weights


# Тестируем на простом примере
# Создаем случайные эмбеддинги (в реальности это будут word embeddings)
seq_len = 5  # 5 слов в предложении
d_model = 64  # размерность эмбеддингов
d_k = 8  # размерность для Q, K

X = np.random.randn(seq_len, d_model)  # наши "эмбеддинги"

output, attention_weights = simple_self_attention(X, d_k)

# Визуализируем attention матрицу
plt.figure(figsize=(8, 6))
plt.imshow(attention_weights, cmap='Blues')
plt.title("Self-Attention Weights (ручная реализация)")
plt.xlabel("Key tokens")
plt.ylabel("Query tokens")
plt.colorbar()
plt.tight_layout()
plt.show()